In [1]:
!pip install pyspark
%matplotlib inline
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)
import pandas as pd

     |████████████████████████████████| 204.2MB 64kB/s 
     |████████████████████████████████| 204kB 35.7MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=8bbbfbb6d44aca4a9405e9a9cdfedf193286c17bfa980268554e5659b2ccf221
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [25]:
import pandas as pd
before = pd.read_csv("tweets.csv")
before

,user,id,location,date,favourites_count,text,retweet
0,PatsFan876,2.999235e+09,NaN,42030,359685,Busting stereotypes here... I'm a truck drivin...,True
1,Yasu_Al_Masih,8.830000e+17,Nazareth,42923,15736,I COULDN'T RESIST POSTING THIS ONE!🤣😂🤣😂🤣🤣🤣 🤦‍♀...,True
2,JBK11,1.503627e+07,Canada,39606,43347,Busting stereotypes here... I'm a truck drivin...,True
3,SoyElG,3.209835e+07,NaN,39919,432,The dismissive hand wave is what’s most disres...,False
4,cynthianatalie,2.110394e+07,Los Angeles,39861,4952,"Inside #DonaldTrump's ""serial bad behavior"" at...",True
...,...,...,...,...,...,...,...
97195,Kimberl93935319,9.230000e+17,NaN,43032,14605,@Amy_Siskind #DEMFOCUS2020 We need to focus re...,True
97196,sharknadoalert,1.280000e+18,NaN,44028,6375,"""Missed opportunities add up to a losing strat...",True
97197,BereniceJB_,4.312218e+08,Orange County,40885,29087,The Gun Lobby is calling on voters to vote for...,True
97198,1989Mckoy,8.440000e+17,NaN,42816,135684,Please vote #registertovote if you have yet to...,False


In [26]:
before.dtypes

user                 object
id                  float64
location             object
date                  int64
favourites_count      int64
text                 object
retweet                bool
dtype: object

In [27]:
# Pandas to Spark
from pyspark.sql.types import *
# mySchema=StructType([StructField("Country", StringType())\
#                       ,StructField("Org", StringType())])

mySchema = StructType([StructField("user", StringType())\
                      ,StructField("id", FloatType())\
                      ,StructField("location", StringType())\
                      ,StructField("date", IntegerType())\
                      ,StructField("favourites_count", StringType())\
                      ,StructField("text", StringType())\
                      ,StructField("retweet", StringType())])
df_before = spark.createDataFrame(before,schema=mySchema)
df_before.show()

+---------------+-------------+--------------------+-----+----------------+--------------------+-------+
|           user|           id|            location| date|favourites_count|                text|retweet|
+---------------+-------------+--------------------+-----+----------------+--------------------+-------+
|     PatsFan876| 2.99923456E9|                 NaN|42030|          359685|Busting stereotyp...|   true|
|  Yasu_Al_Masih| 8.8299999E17|            Nazareth|42923|           15736|I COULDN'T RESIST...|   true|
|          JBK11|  1.5036271E7|              Canada|39606|           43347|Busting stereotyp...|   true|
|         SoyElG|  3.2098352E7|                 NaN|39919|             432|The dismissive ha...|  false|
| cynthianatalie|  2.1103938E7|         Los Angeles|39861|            4952|Inside #DonaldTru...|   true|
|  jaynemikulski|  7.1986381E8| Western Springs, IL|41564|          103327|Is #DonaldTrump a...|   true|
|       Shrinki2|1.21999996E18|     United States, |438

In [28]:
df_before.count()

97200

In [29]:
# drop columns
df_before = df_before.drop('id', 'date')
# change the data type
df_before = df_before.withColumn("favourites_count", df_before['favourites_count'].cast(IntegerType()))
# # drop rows which tweet is retweet
df_before = df_before.filter(df_before['retweet'] == 'false')

In [31]:
# check if there are duplicate rows
if df_before.distinct().count() != df_before.count():
  print('There are duplicate rows.')
else:
  print('No duplicated.')

There are duplicate rows.


In [32]:
# remove duplicate rows
df_before = df_before.dropDuplicates()
df_before.count()

22987

In [36]:
df_before.show(4)

+---------------+--------------------+----------------+--------------------+-------+--------------------+
|           user|            location|favourites_count|                text|retweet|            hashtage|
+---------------+--------------------+----------------+--------------------+-------+--------------------+
|    TerinaKelso|Jacksonville, Flo...|            1865|SAY IT AGAIN FOR ...|  false|[#trump, #maga, #...|
| BadTasteMedia7|      Snellville, Ga|               2|I appreciate Chri...|  false|[#badtastemedia, ...|
|Horizonshealth2|                 NaN|              28|FOODS THAT MAKE Y...|  false|[#DonaldTrump, #T...|
|  maskshirt2021|                 NaN|             125|Without a mask he...|  false|[#COVIDIOTS, #Don...|
+---------------+--------------------+----------------+--------------------+-------+--------------------+
only showing top 4 rows



In [37]:
# extract the hashtage
import re
#extracting hashtags as a means of possible cluster validation  
def extraxt_hashtags(text):
  hashtags = re.findall(r'\B#\w*[a-zA-Z]+\w*', text)
  return hashtags

In [38]:
from pyspark.sql.functions import udf, expr, concat, col
from pyspark.sql import types
func_hash = udf(extraxt_hashtags, types.StringType())
df_before = df_before.withColumn("hashtage", func_hash("text"))

In [39]:
# clean tweet
from string import punctuation
def clean_tweet(tweet):
    # Remove hyperlinks
    tweet = re.sub("https?://[A-Za-z0-9./]*","",tweet)
    # Remove hashtags
    tweet = re.sub(r'#\w*', '', tweet)
    # Remove tickers
    tweet = re.sub(r'\$\w*', '', tweet)
    #@user -> at_user
    tweet = re.sub("@[\w]*","",tweet)
    # To lowercase
    tweet = tweet.lower()
    # Remove Punctuation and split 's, 't, 've with a space for filter
    tweet = re.sub(r'[' + punctuation.replace('@', '') + ']+', ' ', tweet)
    # Remove words: I, a , am, me (2 or less letters)
    tweet = re.sub(r'\b\w{1,2}\b', '', tweet)
    # Remove whitespace (including new line characters)
    tweet = re.sub(r'\s\s+', ' ', tweet)
    # Remove single space remaining at the front of the tweet.
    tweet = tweet.lstrip(' ') 
    # Remove emojis or other. special characters
    tweet = ''.join(c for c in tweet if c <= '\uFFFF') 
    return tweet  

In [40]:
func_clean = udf(clean_tweet, types.StringType())
df_before = df_before.withColumn("clean_text", func_clean("text"))

In [41]:
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import IDF
import requests
from pyspark.ml import Pipeline
from pyspark.ml.feature import CountVectorizer

tokenizer = Tokenizer()\
  .setInputCol("clean_text")\
  .setOutputCol("words")

# stopwords object
# we obtain the stop words from a website
stop_words = requests.get('http://ir.dcs.gla.ac.uk/resources/linguistic_utils/stop_words').text.split()
stop_words[0:10]
sw_filter = StopWordsRemover()\
\
  .setStopWords(stop_words)\
  .setCaseSensitive(False)\
  .setInputCol("words")\
  .setOutputCol("filtered")

# we will remove words that appear in 5 docs or less
cv = CountVectorizer()\
  .setInputCol("filtered")\
  .setOutputCol("tf")

# TF-IDF object
idf = IDF().\
    setInputCol('tf').\
    setOutputCol('tfidf')

# build the pipeline
pipe1 = Pipeline(stages=[tokenizer, sw_filter, cv]).fit(df_before)
tweets_pipe = Pipeline(stages=[pipe1, idf]).fit(df_before)

In [42]:
tweets_before = tweets_pipe.transform(df_before)

In [43]:
tweets_before.show(4)

+---------------+--------------------+----------------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|           user|            location|favourites_count|                text|retweet|            hashtage|          clean_text|               words|            filtered|                  tf|               tfidf|
+---------------+--------------------+----------------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|    TerinaKelso|Jacksonville, Flo...|            1865|SAY IT AGAIN FOR ...|  false|[#trump, #maga, #...|say again for the...|[say, again, for,...|  [say, people, row]|(20493,[9,50,3189...|(20493,[9,50,3189...|
| BadTasteMedia7|      Snellville, Ga|               2|I appreciate Chri...|  false|[#badtastemedia, ...|appreciate christ...|[appreciate, chri...|[apprecia

In [44]:
tweets_before.toPandas().to_csv('before.csv')

## After

In [47]:
df_after = pd.read_csv("tweet_after.csv")
df_after

,user,id,location,date,favourites_count,text,retweet
0,Hskers62,9.081270e+17,United States,9/14/17,197954.0,Donald Trump is the only president in history ...,True
1,JMZJEWELRY,8.826030e+17,Here and there,7/5/17,3652.0,Joe Biden got 306 Electoral College votes by w...,True
2,lrogers66821,1.004390e+18,"Cincinnati, OH",6/6/18,369957.0,"Deutsche Bank, which funded Auschwitz, Donald ...",True
3,LizNYC66,8.927390e+17,"Manhattan, NY",8/2/17,33175.0,The Biden administration is sure to come under...,True
4,thia89837769,1.214570e+18,"Ottawa, Ontario",1/7/20,5580.0,Donald Trump is the only president in history ...,True
...,...,...,...,...,...,...,...
45389,Richardson9Mike,1.219890e+18,NaN,1/22/20,21.0,Should Nancy go to jail? #Elections2020 #Elect...,False
45390,suzyqbooks,3.720740e+07,"Salt Lake City, UT",5/2/09,8759.0,"VOTE UPDATE:\n\nGeorgia: Biden +12,337 \nArizo...",True
45391,MSZ006FD3S3121,1.041590e+18,NaN,9/17/18,47294.0,"VOTE UPDATE:\n\nGeorgia: Biden +12,337 \nArizo...",True
45392,FjellVeitje,1.297540e+18,Noreg,8/23/20,2731.0,I think this is a vote we all can agree on... ...,True


In [48]:
df_after.dtypes

user                 object
id                  float64
location             object
date                 object
favourites_count    float64
text                 object
retweet              object
dtype: object

In [49]:
from pyspark.sql.types import *
# mySchema=StructType([StructField("Country", StringType())\
#                       ,StructField("Org", StringType())])

mySchema = StructType([StructField("user", StringType())\
                      ,StructField("id", FloatType())\
                      ,StructField("location", StringType())\
                      ,StructField("date", StringType())\
                      ,StructField("favourites_count", StringType())\
                      ,StructField("text", StringType())\
                      ,StructField("retweet", StringType())])
tweets_after = spark.createDataFrame(df_after,schema=mySchema)
tweets_after.show(5)

+------------+-------------+---------------+-------+----------------+--------------------+-------+
|        user|           id|       location|   date|favourites_count|                text|retweet|
+------------+-------------+---------------+-------+----------------+--------------------+-------+
|    Hskers62| 9.0812699E17|  United States|9/14/17|        197954.0|Donald Trump is t...|   true|
|  JMZJEWELRY|   8.82603E17|Here and there | 7/5/17|          3652.0|Joe Biden got 306...|   true|
|lrogers66821|1.00438999E18| Cincinnati, OH| 6/6/18|        369957.0|Deutsche Bank, wh...|   true|
|    LizNYC66| 8.9273898E17|  Manhattan, NY| 8/2/17|         33175.0|The Biden adminis...|   true|
|thia89837769|1.21457002E18|Ottawa, Ontario| 1/7/20|          5580.0|Donald Trump is t...|   true|
+------------+-------------+---------------+-------+----------------+--------------------+-------+
only showing top 5 rows



In [50]:
tweets_after.count()

45394

In [51]:
# drop columns
tweets_after = tweets_after.drop('id', 'date')
# change the data type
tweets_after = tweets_after.withColumn("favourites_count", tweets_after['favourites_count'].cast(IntegerType()))
# # drop rows which tweet is retweet
tweets_after = tweets_after.filter(tweets_after['retweet'] == 'false')

In [52]:
# check if there are duplicate rows
if tweets_after.distinct().count() != tweets_after.count():
  print('There are duplicate rows.')
else:
  print('No duplicated.')

There are duplicate rows.


In [53]:
# remove duplicate rows
tweets_after = tweets_after.dropDuplicates()
tweets_after.count()

15860

In [54]:
tweets_after = tweets_after.withColumn("hashtage", func_hash("text"))

In [55]:
tweets_after = tweets_after.withColumn("clean_text", func_clean("text"))

In [56]:
pipe2 = Pipeline(stages=[tokenizer, sw_filter, cv]).fit(tweets_after)
tweets_pipe_2 = Pipeline(stages=[pipe2, idf]).fit(tweets_after)

In [57]:
tweets_after_df = tweets_pipe_2.transform(tweets_after)

In [58]:
tweets_after_df.show(5)

+---------------+----------------+----------------+--------------------+-------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|           user|        location|favourites_count|                text|retweet|    hashtage|          clean_text|               words|            filtered|                  tf|               tfidf|
+---------------+----------------+----------------+--------------------+-------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|nicholasmcardle|White House -TBC|             418|@realDonaldTrump ...|  false|          []|trump english can...|[trump, english, ...|[trump, english, ...|(17372,[0,4,6,10,...|(17372,[0,4,6,10,...|
|     seanieinaz|             NaN|              38|@realDonaldTrump ...|  false|          []|can donald trump ...|[can, donald, tru...|[donald, trump, t...|(17372,[0,4,39,64...|(17372,[0,4,39,64...|
|    

In [59]:
tweets_after_df.toPandas().to_csv('after.csv')